In [1]:
#current_dir = pwd()
#app_dir = "/app"
#cd(app_dir)

#push!(LOAD_PATH, app_dir)
#push!(LOAD_PATH, current_dir)
push!(LOAD_PATH, "/home/jberez/Projects/BostonHousing/docker-parallel")

using Gen
using Distributions
using LinearAlgebra
using Flux
using Random
using Distances
using JLD
using Serialization
using StatsBase
using BNN

include("docker-parallel/NUTS.jl")
include("docker-parallel/RJNUTS.jl")
include("docker-parallel/utils.jl")
include("docker-parallel/proposals.jl")
include("docker-parallel/LoadData.jl");

In [2]:
println(Threads.nthreads())

4


In [ ]:
function RJNUTS_parallel(trace, chain)
    
    (trace, a_acc) = layer_parameter(trace)
    trace  = gibbs_hyperparameters(trace)
    trace  = gibbs_noise(trace)
    (trace, w_acc)  = layer_nuts(trace)
    println("Chain $chain Iter $i : $(get_score(trace))")

    return trace, a_acc, w_acc
end

In [ ]:
#MAIN SIMULATION

include("docker-parallel/RJNUTS.jl")
println("Packages Loaded")

filename = "testparallel.jld"
ITERS = 20
CHAINS = Threads.nthreads()

#---------------
#Hyperparameters
#---------------

#NUTS hyperparameters
Δ_max = 10
acc_prob = 0.65
m=1
m2=1

#Network hyperparameters
k_real = 4 #Number of hidden nodes per layer
k_vector = [0.0 for i=1:k_real]
k_vector[k_real] = 1.0

#Layer hyperparameters
l_range = 8 #Maximum number of layers in the network
l_list = [Int(i) for i in 1:l_range]
l_real = 1;

#Hyperprior Hyperparameters
αᵧ = 1 #Regression Noise Shape
βᵧ = 1 #Regression Noise Scale/Rate
α₁ = 1 #Input Weights, Biases Shape
β₁ = 1 #Input Weights, Biases Scale/Rate
α₂ = 1 #Hidden & Output Weights Shape
β₂ = k_real; #Hidden & Output Weights Scale

obs_master = choicemap()::ChoiceMap
obs_master[:y] = y_train
obs = obs_master;

#-----------
#Parallelize
#-----------

println("Initializing Traces")
println("-------------------")

traces = [[] for i=1:CHAINS]
a_acc = [[] for i=1:CHAINS]
w_acc = [[] for i=1:CHAINS]

for i=1:CHAINS
    obs[:l] = ((i-1)%8 + 1)
    (new_start,) = generate(interpolator, (x_train,), obs)
    push!(traces[i],new_start)
end

active_trace = [traces[i][1] for i=1:CHAINS]
a_active = [[] for i=1:CHAINS]
w_active = [[] for i=1:CHAINS]

obs = obs_master

#--------------
#Run Inference
#--------------
#cd(current_dir)
println("Beginning Inference")
println("-------------------")
#traces, scores = RJNUTS(trace, ITERS)

for i2=1:ITERS
    Threads.@threads for i=1:CHAINS
        active_trace[i],_,_ = @inbounds RJNUTS_parallel(traces[i][i2], i, i2)
        push!(traces[i],active_trace[i])
        push!(a_acc[i],a_active[i])
        push!(w_acc[i],w_active[i])
    end
    flush(stdout)
    serialize(filename, traces)
end

#serialize(filename, traces)

Packages Loaded
Initializing Traces
-------------------
Beginning Inference
-------------------
********** Accepted: 2 **********
Chain 1 Iter 1 : -662.7687151600658
Chain 2 Iter 2 : -419.79986840982235
********** Accepted: 5 **********
********** Accepted: 2 **********
Chain 3 Iter 3 : -501.45128634074626
Chain 4 Iter 4 : -538.3215519198154
Chain 2 Iter 2 : -422.41751906726427
Chain 3 Iter 3 : -498.23116012169965
********** Accepted: 3 **********
********** Accepted: 4 **********
Chain 1 Iter 1 : -491.57956305596326
Chain 4 Iter 4 : -482.2711577297891
Chain 2 Iter 2 : -406.19879206394853
Chain 3 Iter 3 : -487.2386255556628
Chain 1 Iter 1 : -470.7337829716173


In [4]:
ls = [[traces[i][j][:l] for j=1:length(traces[i])] for i=1:CHAINS]

4-element Array{Array{Int64,1},1}:
 [1, 1, 2, 2]
 [2, 2, 1, 2]
 [3, 2, 1, 2]
 [4, 4, 4, 3]

In [5]:
recovered = deserialize("testparallel.jld");

In [6]:
recovered[1]

4-element Array{Any,1}:
 Gen.DynamicDSLTrace{DynamicDSLFunction{Any}}(DynamicDSLFunction{Any}(Dict{Symbol,Any}(), Dict{Symbol,Any}(), Type[Any], false, Union{Nothing, Some{Any}}[nothing], BNN.##interpolator#253, Bool[0], false), Trie{Any,Gen.ChoiceOrCallRecord}(Dict{Any,Gen.ChoiceOrCallRecord}((:k, 2) => Gen.ChoiceOrCallRecord{Int64}(1, 0.0, NaN, true),(:τᵦ, 2) => Gen.ChoiceOrCallRecord{Float64}(0.632419964960025, -0.632419964960025, NaN, true),(:b, 2) => Gen.ChoiceOrCallRecord{Array{Float64,1}}([1.212476089634327], -1.6128990827520941, NaN, true),(:b, 1) => Gen.ChoiceOrCallRecord{Array{Float64,1}}([-0.16727771877508316, -1.3670719183021043, 1.0795209327417044, 0.651359259837773], -6.5105497195079955, NaN, true),:l => Gen.ChoiceOrCallRecord{Int64}(1, -2.0794415416798357, NaN, true),(:W, 2) => Gen.ChoiceOrCallRecord{Array{Float64,1}}([-0.5545792270094025, 0.5588241354163679, 0.1854093242132301, 1.0007588541134396], -3.915453383400587, NaN, true),(:k, 1) => Gen.ChoiceOrCallRecord{Int64}(